In [ ]:
from ultralytics import YOLO
import supervision as sv
import cv2
import numpy as np
import os
import pytesseract
import re

# Load your trained YOLOv8 model
model = YOLO('../model/yolo.pt')  # Path to your trained weights

# Define the directories for input images and cropped plates
image_folder = "../images"  # Replace with your folder path
output_folder = "test_output"  # Folder to save cropped plate images
os.makedirs(output_folder, exist_ok=True)

# Tesseract configuration for optimal performance
custom_config = r'--oem 3 --psm 6'
cleaned_output_file = "all_cleaned_texts.txt"

# Function to clean unwanted symbols and trailing characters
def clean_text(text):
    cleaned = re.sub(r'[^a-zA-Z0-9\u1780-\u17FF\s-]', '', text)
    cleaned = re.sub(r'\s+[ន]*$', '', cleaned.strip())
    return cleaned

# Function to extract text from an image using Tesseract
def extract_text_from_image(image):
    image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )
    raw_text = pytesseract.image_to_string(thresh, config=custom_config, lang='khm+eng')
    lines = raw_text.strip().split('\n')
    cleaned_lines = [clean_text(line) for line in lines if clean_text(line)]
    return "\n".join(cleaned_lines)

# Process each image in the input folder
with open(cleaned_output_file, "w", encoding="utf-8") as cleaned_f:
    for image_file in os.listdir(image_folder):
        if image_file.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(image_folder, image_file)
            image = cv2.imread(image_path)

            # Perform detection using YOLOv8
            results = model(image)[0]
            xyxy = results.boxes.xyxy.numpy()
            confidences = results.boxes.conf.numpy()
            class_ids = results.boxes.cls.numpy().astype(int)

            detections = sv.Detections(xyxy=xyxy, confidence=confidences, class_id=class_ids)
            bounding_box_annotator = sv.BoxAnnotator()
            annotated_image = bounding_box_annotator.annotate(scene=image, detections=detections)

            # Iterate over detected plates and extract text
            for idx, box in enumerate(xyxy):
                x1, y1, x2, y2 = map(int, box)
                plate_image = image[y1:y2, x1:x2]  # Crop the plate from the image
                cropped_path = os.path.join(output_folder, f"{image_file}_plate_{idx}.jpg")
                cv2.imwrite(cropped_path, plate_image)  # Save the cropped plate image

                # Extract text from the cropped plate
                cleaned_text = extract_text_from_image(plate_image)
                cleaned_f.write(f"\n[Cleaned Text from {image_file}_plate_{idx}]:\n{cleaned_text}\n")

            # Display the annotated image
            sv.plot_image(image=annotated_image, size=(16, 16))

print(f"Processing completed. Extracted texts are saved in {cleaned_output_file}.")



In [ ]:
import cv2
import pytesseract
import imutils

# Optional: Set the Tesseract executable path if you are on Windows
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_license_plate_text(image_path):
    # Step 1: Load the image and resize for consistency
    image = cv2.imread(image_path)
    image = imutils.resize(image, width=600)

    # Step 2: Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 3: Apply bilateral filter to reduce noise while keeping edges sharp
    filtered = cv2.bilateralFilter(gray, 11, 17, 17)

    # Step 4: Use adaptive thresholding for better text visibility
    thresh = cv2.adaptiveThreshold(filtered, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY, 11, 2)

    # Step 5: Detect edges using Canny
    edges = cv2.Canny(thresh, 30, 200)

    # Step 6: Find contours and sort by area
    contours = cv2.findContours(edges.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    # Step 7: Loop through contours to find a 4-point contour (possible plate)
    plate = None
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

        # Check for 4-cornered contour (rectangle or trapezoid)
        if len(approx) == 4:
            plate = approx
            break

    # Step 8: Extract the region of interest (ROI) if plate-like contour is found
    if plate is not None:
        x, y, w, h = cv2.boundingRect(plate)
        roi = gray[y:y+h, x:x+w]

        # Step 9: Apply Otsu's threshold to improve text visibility
        roi = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

        # Optional: Display the ROI to verify clarity
        cv2.imshow("ROI", roi)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # Step 10: Perform OCR using Tesseract with Khmer + English languages
        config = '--psm 7'  # Assume a single line of text
        text = pytesseract.image_to_string(roi, config=config, lang='khm+eng')

        return text.strip()
    else:
        return "License plate not detected."

# Test the function with your image path
image_path = 'images/71039975-Phnom-Penh-license-plate-Phnom-Penh-Phnom-Penh-Cambodia-Asia.jpg'
text = extract_license_plate_text(image_path)
print("Detected Text:", text)
